In [7]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

In [8]:
def lables2id(lables):
    target_id = []
    target_lables = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
    for lable in lables:
        target_id.append(target_lables.index(lable))
    return target_id

class CLDataset(Dataset):
  def __init__(self, path):
    data = pd.read_csv(path)
    x_data = data.drop(['id', 'target'], axis=1)
    labels = data['target']
    self.x_data = torch.from_numpy(x_data.to_numpy()).float()
    self.y_data = torch.tensor(lables2id(labels)).long()
    self.len = data.shape[0]

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]
  
  def __len__(self):
    return self.len

In [9]:
dataset = CLDataset('../dataset/otto-group-product-classification-challenge/train.csv')
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), len(dataset)-int(len(dataset)*0.8)])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [10]:
for X, y in train_loader:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X: torch.Size([64, 93])
Shape of y: torch.Size([64]) torch.int64


In [11]:
for X, y in test_loader:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X: torch.Size([64, 93])
Shape of y: torch.Size([64]) torch.int64


In [12]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.linear1 = torch.nn.Linear(93, 64)
    self.linear2 = torch.nn.Linear(64, 32)
    self.linear3 = torch.nn.Linear(32, 16)
    self.linear4 = torch.nn.Linear(16, 9)
    self.relu = torch.nn.ReLU()

  def forward(self, x):
    x = self.relu(self.linear1(x))
    x = self.relu(self.linear2(x))
    x = self.relu(self.linear3(x))
    x = self.linear4(x)
    return x
   
model = Net()

In [13]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
  running_loss = 0.0
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 100 == 99:
      print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 100))
      running_loss = 0.0
    

In [14]:
def test():
  correct = 0
  total = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = model(data)
      _, predicted = torch.max(output.data, 1)
      total += target.size(0)
      correct += (predicted == target).sum().item()
  
  print('Accuracy of the network on the test data: %d %%' % (100 * correct / total))

In [15]:
for epoch in range(10):
  train(epoch)
  test()

[1,   100] loss: 1.989
[1,   200] loss: 1.614
[1,   300] loss: 1.263
[1,   400] loss: 1.079
[1,   500] loss: 0.965
[1,   600] loss: 0.898
[1,   700] loss: 0.825
Accuracy of the network on the test data: 73 %
[2,   100] loss: 0.757
[2,   200] loss: 0.763
[2,   300] loss: 0.748
[2,   400] loss: 0.725
[2,   500] loss: 0.701
[2,   600] loss: 0.743
[2,   700] loss: 0.680
Accuracy of the network on the test data: 72 %
[3,   100] loss: 0.670
[3,   200] loss: 0.698
[3,   300] loss: 0.691
[3,   400] loss: 0.681
[3,   500] loss: 0.676
[3,   600] loss: 0.694
[3,   700] loss: 0.675
Accuracy of the network on the test data: 75 %
[4,   100] loss: 0.657
[4,   200] loss: 0.670
[4,   300] loss: 0.654
[4,   400] loss: 0.674
[4,   500] loss: 0.669
[4,   600] loss: 0.643
[4,   700] loss: 0.641
Accuracy of the network on the test data: 75 %
[5,   100] loss: 0.629
[5,   200] loss: 0.641
[5,   300] loss: 0.624
[5,   400] loss: 0.643
[5,   500] loss: 0.666
[5,   600] loss: 0.639
[5,   700] loss: 0.640
Accurac